In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


In [2]:
BASE_URL = "https://books.toscrape.com/catalogue/"


In [3]:
def parse_book_details(book_url):
    response = requests.get(book_url)
    soup = BeautifulSoup(response.content, 'html.parser')

    upc = soup.find('table', class_='table').find_all('tr')[0].find_all('td')[0].text
    return upc



In [4]:
def parse_page(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    books = []
    for book in soup.find_all('article', class_='product_pod'):
        book_name = book.h3.a['title']
        price = book.find('p', class_='price_color').text[1:]
        book_url = BASE_URL + book.h3.a['href']

        book_upc = parse_book_details(book_url)
        books.append({
            'id': book_upc,
            'book_name': book_name,
            'price': price,
            'url': book_url
        })
    return books

In [5]:
def scrape_books(max_books=1000):
    all_books = []
    page = 1
    while len(all_books) < max_books:
        url = f"https://books.toscrape.com/catalogue/page-{page}.html"
        books = parse_page(url)
        if not books:
            break
        all_books.extend(books)
        page += 1

    return all_books[:max_books]

In [7]:
books_data = scrape_books()

df = pd.DataFrame(books_data)
df.to_csv('books.csv', index=False)

print("Файл books.csv создан!")

Файл books.csv создан!
